In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 M

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=7)

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/59.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)
    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 20)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        logging_dir="./logs",  
        logging_steps=10, 
        save_steps=500,
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",  
        logging_first_step=True,  
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    print("Starting training...")
    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:")
print(study.best_trial)

[I 2024-12-01 18:04:34,139] A new study created in memory with name: no-name-0222862b-c32e-4dfb-a499-62fb1b4aa9e0
<ipython-input-10-747f1883d862>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-747f1883d862>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)


Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 9
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 585


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.652100,0.641178,0.000000,1.000000,0.000000,0.000000
2,0.599000,0.592053,0.000000,1.000000,0.000000,0.000000
3,0.564600,0.555377,0.000000,1.000000,0.000000,0.000000
4,0.530600,0.527685,0.000000,1.000000,0.000000,0.000000
5,0.511300,0.507684,0.000000,1.000000,0.000000,0.000000
6,0.497900,0.493723,0.000000,1.000000,0.000000,0.000000
7,0.485500,0.484699,0.000000,1.000000,0.000000,0.000000
8,0.488900,0.479627,0.000000,1.000000,0.000000,0.000000
9,0.481100,0.477988,0.000000,1.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


...

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.001200,0.452477,0.692308,0.783223,0.745819,0.761654
1,0.007200,0.466141,0.703846,0.801766,0.735786,0.760865
2,0.008600,0.493529,0.684615,0.771606,0.729097,0.739784
3,0.001800,0.463223,0.700000,0.796979,0.732441,0.758748
4,0.000800,0.458465,0.707692,0.777572,0.752508,0.763008
5,0.001000,0.470276,0.688462,0.788491,0.729097,0.752029
6,0.000800,0.458177,0.696154,0.782051,0.735786,0.754857
7,0.000800,0.456390,0.692308,0.787725,0.735786,0.755761
8,0.000800,0.475617,0.665385,0.773215,0.715719,0.736484
9,0.004400,0.478987,0.665385,0.774356,0.719064,0.738476


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


[I 2024-12-01 18:34:07,707] Trial 19 finished with value: 0.7498955172153927 and parameters: {'learning_rate': 3.0666349642617465e-05, 'batch_size': 16, 'num_train_epochs': 11, 'weight_decay': 0.009563083630882568}. Best is trial 18 with value: 0.7602403360567198.


Best trial:
FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.7602403360567198], datetime_start=datetime.datetime(2024, 12, 1, 18, 31, 54, 494059), datetime_complete=datetime.datetime(2024, 12, 1, 18, 33, 10, 272012), params={'learning_rate': 2.888231039965605e-05, 'batch_size': 4, 'num_train_epochs': 11, 'weight_decay': 0.01929556880988707}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=20, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=18, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

print("Starting training...")
start_time = time.time()  

trainer.train()

end_time = time.time() 
training_duration = end_time - start_time  

# Print the training duration
print(f"Training completed in {training_duration:.2f} seconds.")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 11
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1430


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.484730,0.692308,0.775827,0.729097,0.744146
2,No log,0.519850,0.688462,0.779963,0.725753,0.745794
3,No log,0.487079,0.703846,0.780691,0.735786,0.750682
4,0.002400,0.522243,0.680769,0.773223,0.712375,0.729849
5,0.002400,0.507484,0.680769,0.781162,0.725753,0.744547
6,0.002400,0.491256,0.696154,0.791689,0.725753,0.750432
7,0.002400,0.486379,0.692308,0.782738,0.725753,0.747722
8,0.000500,0.491084,0.696154,0.789770,0.729097,0.753370
9,0.000500,0.500096,0.700000,0.788901,0.729097,0.750961
10,0.000500,0.505261,0.696154,0.791448,0.729097,0.751734


***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch

Training completed in 73.92 seconds.


In [13]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 260
  Batch size = 8


### Model Saving

In [14]:
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_pharo'
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo')
tokenizer.save_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo')
tokenizer = AutoTokenizer.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer')

Saving model checkpoint to ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo
Configuration saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/config.json
Model weights saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 1, 1, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 1, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 1, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 0, 1, 0]) array([0, 0, 0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([0, 0, 0, 0, 1, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 289
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.6505190311418685, 'precision': 0.69406080261562, 'recall': 0.6943521594684385, 'f1': 0.6882669637792822}
Inference Time: 0.69 seconds


In [18]:
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_pharo.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_pharo'
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer'

  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/pytorch_model.bin (deflated 7%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/training_args.bin (deflated 51%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/config.json (deflated 54%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/tokenizer_config.json (deflated 41%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/tokenizer.json (deflated 71%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/vocab.txt (deflated 53%)


In [19]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_huawei-noah_TinyBERT_General_4L_312D_pharo.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo/"
!cp best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo/best_model_huawei-noah_TinyBERT_General_4L_312D_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo/best_model_huawei-noah_TinyBERT_General_4L_312D_pharo.zip
replace ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/pytorch_model.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/training_args.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_pharo_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer.zip
replace ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer/tokenizer_config.json  
  inflating: ./best_model_huawei-noah_TinyBERT_Gen

In [28]:
best_model_huawei_noah_TinyBERT_General_4L_312D_pharo = 'best_model_huawei-noah_TinyBERT_General_4L_312D_pharo'
best_model_huawei_noah_TinyBERT_General_4L_312D_pharo_tokenizer = 'best_model_huawei-noah_TinyBERT_General_4L_312D_pharo_tokenizer'

In [29]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_pharo)
tokenizer = AutoTokenizer.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_pharo_tokenizer)

loading configuration file best_model_huawei-noah_TinyBERT_General_4L_312D_pharo/config.json
Model config BertConfig {
  "_name_or_path": "best_model_huawei-noah_TinyBERT_General_4L_312D_pharo",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",

In [30]:
text = "Logits from the model output."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

SequenceClassifierOutput(loss=None, logits=tensor([[-8.3854, -8.3199, -8.9910, -9.3650, -9.3535,  7.3401, -8.9232]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 6
